In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import Point, wkb, get_coordinates, geometry
import matplotlib.pyplot as plt
import sqlalchemy
from zoneinfo import ZoneInfo
import osmnx as ox
import warnings
from mappymatch.constructs.trace import Trace
from mappymatch.maps.nx.nx_map import NxMap
from mappymatch.matchers.lcss.lcss import LCSSMatcher
from mappymatch.matchers.line_snap import LineSnapMatcher
from mappymatch.maps.nx.readers.osm_readers import *

## settings

In [ ]:
local_crs = 3006
osm_crs = 4326
graph_crs = 32634

row_limit_lines = 1000000
n_batches = 1
n_batch = 1
city_abbr = 'gbg'

mapmatching_graph = 'custom'       # custom or osm
match_algorithm = 'LCSS'        # LCSS, LineSnap

write_to_table = True
if_exists = 'replace' # replace or append
plot = True

## read trajectories

In [ ]:
# read from database
url = sqlalchemy.URL.create(
    "postgresql+psycopg", port=5432,
    host="host", database="database", username="username")
engine = sqlalchemy.create_engine(url)

In [ ]:
traj_table_name = '{}_trajectory_lines_2024'.format(city_abbr)

In [ ]:
if n_batches > 1:
    sql_query_lines = """
    WITH
    NumberedRows AS (
        SELECT *, ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS RowNum, COUNT(*) OVER () AS TotalRows
        FROM trajectories.{0}
        ),
    Percentiles AS (
        SELECT *, NTILE({1}) OVER (ORDER BY RowNum) AS Percentile
        FROM NumberedRows
        )
    SELECT *
    FROM Percentiles
    WHERE Percentile = {2};
    """.format(traj_table_name, n_batches, n_batch)
else:
    sql_query_lines = """
    SELECT *
    FROM trajectories.{0}
    ORDER BY RANDOM()
    LIMIT {1};
    """.format(traj_table_name, row_limit_lines)

with engine.connect() as conn_flowsense:
    with conn_flowsense.execute(sqlalchemy.text(sql_query_lines)) as cursor:
        df_lines = pd.read_sql(sql_query_lines, con=conn_flowsense)

In [ ]:
traj_lines = gpd.GeoDataFrame(df_lines, geometry=df_lines['geometry'].apply(wkb.loads), crs=local_crs)

In [ ]:
traj_lines['start_timestamp_se'] = traj_lines['start_timestamp_se'].dt.tz_convert(ZoneInfo("Europe/Stockholm"))
traj_lines['end_timestamp_se'] = traj_lines['end_timestamp_se'].dt.tz_convert(ZoneInfo("Europe/Stockholm"))

In [ ]:
len(traj_lines)

## read custom network (if applicable)

In [ ]:
if mapmatching_graph == 'custom':

    sql_query_edges = """
        SELECT *
        FROM road_network.trafikverket_edges_{};
        """.format(city_abbr)

    sql_query_nodes = """
        SELECT *
        FROM road_network.trafikverket_nodes_{};
        """.format(city_abbr)

    with engine.connect() as conn_flowsense:
        with conn_flowsense.execute(sqlalchemy.text(sql_query_lines)) as cursor:
            df_edges = pd.read_sql(sql_query_edges, con=conn_flowsense)
            df_nodes = pd.read_sql(sql_query_nodes, con=conn_flowsense)

    custom_edges = gpd.GeoDataFrame(df_edges, geometry=df_edges['geometry'].apply(wkb.loads), crs=local_crs)
    custom_nodes = gpd.GeoDataFrame(df_nodes, geometry=df_nodes['geometry'].apply(wkb.loads), crs=local_crs)

    custom_edges.to_crs(graph_crs, inplace=True)
    custom_nodes.to_crs(graph_crs, inplace=True)

## mapmatching with mappymatch library

https://nrel.github.io/mappymatch/lcss-example.html

Zhu, Lei, Jacob R. Holden, and Jeffrey D. Gonder. "Trajectory Segmentation Map-Matching Approach for Large-Scale, High-Resolution GPS Data." Transportation Research Record: Journal of the Transportation Research Board 2645 (2017): 67-75.

In [ ]:
bbox = gpd.GeoDataFrame(index=[0], geometry=[geometry.box(*traj_lines.total_bounds)], crs=local_crs)

In [ ]:
if mapmatching_graph == 'osm':

    # projected graph of drivable roads
    # combining OSM network_type filter 'all' with excluding dedicated pedestrian/bikeways and steps
    custom_filter = """
        ["highway"]["area"!~"yes"]["highway"!~"abandoned|construction|no|planned|platform|proposed|raceway|razed'
        '|footway|pedestrian|steps|cycleway"]"""

    G = ox.project_graph(ox.graph_from_polygon(bbox.to_crs(osm_crs).geometry[0], network_type='drive', custom_filter=custom_filter, retain_all=True))
    nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

elif mapmatching_graph == 'custom':
    pass

    custom_edges.set_index(['u', 'v', 'key'], inplace=True)
    custom_nodes.set_index(['osmid'], inplace=True)

    # create and simplify the graph
    G = ox.graph_from_gdfs(custom_nodes, custom_edges)
    G = ox.simplify_graph(G, edge_attrs_differ=['straight_line'])

    # get final nodes and edges gdfs after simplifying
    nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

    edges.drop(columns='straight_line', inplace=True)

In [ ]:
if plot:
    ox.plot_graph(G)

In [ ]:
nx_map = NxMap(parse_osmnx_graph(G, network_type=NetworkType.DRIVE, xy=True))

In [ ]:
if match_algorithm == 'LCSS':
    matcher = LCSSMatcher(nx_map, similarity_cutoff=0.95)
elif match_algorithm == 'LineSnap':
    matcher = LineSnapMatcher(nx_map)

In [ ]:
def match_with_mappymatch(row, crs=3857, warn=False):

    coordinates = get_coordinates(row.geometry)
    points = [(x, y) for x, y in zip(coordinates[:, 0], coordinates[:, 1])]
    gdf_points = gpd.GeoDataFrame(geometry=[Point(xy) for xy in points], crs=local_crs).to_crs(crs)

    trace = Trace.from_geo_dataframe(gdf_points)
    match_result = matcher.match_trace(trace)

    if len(match_result.path_to_dataframe()):
        gdf = match_result.path_to_geodataframe().to_crs(local_crs)
    elif len(match_result.matches_to_dataframe()) and (type(matcher) == LineSnapMatcher):
        gdf = match_result.matches_to_geodataframe().to_crs(local_crs)
    else:
        gdf = gpd.GeoDataFrame()

    if len(gdf):
        from_to = list(zip(
            gdf.origin_junction_id.to_list(),
            gdf.destination_junction_id.to_list()))
        road_ids = list(gdf.road_id)
        dissolved = gdf.dissolve()
        return dissolved.geom[0], from_to, road_ids

    else:
        if warn:
            (print('Matching failed for trajectory with id '+str(row.traj_id)))
        return row.geometry, None, None

In [ ]:
to_match = traj_lines.copy()
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    to_match[['geometry', 'from_to', 'road_ids']] = to_match.apply(lambda row: match_with_mappymatch(row), axis=1, result_type='expand')
mapmatched = to_match[to_match.from_to.notnull()].copy()
unmatched = to_match[to_match.from_to.isnull()].copy()

In [ ]:
print("{} out of {} ({}%) trajectories were successfully matched".format(len(mapmatched), len(to_match), round(100*(len(mapmatched)/len(to_match)), 2)))

In [ ]:
if plot:
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(12,12))

    edges.to_crs(local_crs).plot(ax=axs, color='#f6f6f6', linewidth=0.5, zorder=1)
    if len(mapmatched): mapmatched.plot(ax=axs, color='red', linewidth=1, zorder=2, alpha=0.05)
    if len(unmatched): unmatched.plot(ax=axs, color='cyan', linewidth=1, zorder=2, alpha=0.05)

    plt.axis('off')

    plt.show()

In [ ]:
len(mapmatched)

## write to table

In [ ]:
# write to database
if write_to_table:

    url_flowsense = sqlalchemy.URL.create(
        "postgresql+psycopg", port=5432,
        host="host", database="database", username="username")
    engine_flowsense = sqlalchemy.create_engine(url_flowsense)

    mapmatched.to_postgis(
        name=traj_table_name + '_{}_random2'.format(match_algorithm.lower()),
        con=engine_flowsense,
        schema='trajectories',
        if_exists=if_exists,
        index=False)

    unmatched.to_postgis(
        name=traj_table_name + '_{}_random2_unmatched'.format(match_algorithm.lower()),
        con=engine_flowsense,
        schema='trajectories',
        if_exists=if_exists,
        index=False)